In [3]:
!pip install openai tiktoken

In [4]:
import getpass, openai, os
client = getpass.getpass(prompt="OPENAI - KEY: ")
openai.apikey = client
os.environ["OPENAI_API_KEY"] = client


OPENAI - KEY: ··········


In [14]:
from openai import OpenAI
client = OpenAI()

In [41]:
import requests
import json

movieSearchResults = []

for page in [1, 2]:
    omdbApiRequest = requests.get(f"https://www.omdbapi.com/?s=saw&page={page}&apikey=10ebe91b")
    movieResultsJson = json.loads(omdbApiRequest.text)
    movieSearchResults += movieResultsJson["Search"]

movieDetails = []

for movie in movieSearchResults:
    movieId = movie["imdbID"]
    movieSearch = requests.get(f"https://www.omdbapi.com/?i={movieId}&apikey=10ebe91b")
    movieDetails += [json.loads(movieSearch.text)]

In [42]:
clases = [pelicula["Type"] for pelicula in movieSearchResults]

# Crear un conjunto a partir de la lista de clases
clases_unicas = set(clases)
clases_unicas

{'movie', 'series'}

In [43]:
moviesString = ""
for mediaType in list(clases_unicas):
    moviesString += f"\n{mediaType}:\n"
    for movie in movieDetails:
        if movie["Type"] == mediaType:
            moviesString += f"{movie['Title']}\n"

print(moviesString)


movie:
Saw
Saw II
Saw III
The Texas Chain Saw Massacre
Saw IV
I Saw the Devil
Saw V
Saw VI
Saw 3D
Saw X
Saw
I Saw the Light
The Last Time I Saw Paris
What Josiah Saw
I Saw What You Did
Saw Rebirth
Who Saw Her Die?
Anohana the Movie: The Flower We Saw That Day (Ano hi mita hana no namae wo bokutachi wa mada shiran
The Last Thing Mary Saw

series:
Anohana: The Flower We Saw That Day



In [44]:
def createModeration(query):
    moderation = client.moderations.create(input = query)
    results = moderation.results
    categories = zip(results[0].categories, results[0].category_scores)
    flaggedStatus = results[0].flagged
    return categories, flaggedStatus

In [45]:

system_message = f"""
You will be provided with queries about media. \
The user query will be delimited with \
{DELIMITER} characters.

Output a python list of objects, where each object has \
the following format:
    'Type': <one of 'series', 'game', 'movie'>,
OR
    'Titles': <a list of media titles that must \
    be found in the allowed titles below>

Where the types of media and the titles must be found in \
the customer service query.
Whenever a title is mentioned, your output must associate it with its \
respective type as in the allowed titles list below.
If no titles or types are found, output an \
empty list.


Allowed media:

{moviesString}

First, check whether the title or media type are explicitly in the query.

If not, check whether they are implied.



Only output the list of objects, with nothing else.
"""

user_query = """
What can you tell me about the saw movies?
"""



In [46]:
DELIMITER = "####"

def getCompletionFromMessages(
        query,
        messages,
        model = "gpt-4",
        temperature = 0,
        delimiter = DELIMITER
):
    query = f"{DELIMITER}{query}{DELIMITER}"
    messages += [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        messages = messages,
        temperature = temperature,
        model = model
    )
    responseContent = response.choices[0].message.content
    messages += [{"content": responseContent, "role": "assistant"}]
    return messages

In [47]:
messages =  [{'role':'system', 'content': system_message}]

messages = getCompletionFromMessages(user_query, messages)

messages[-1]["content"]

"[{'Type': 'movie'}]"

In [48]:
mediaResultsData = json.loads(messages[-1]["content"].replace("'", "\""))

In [49]:
def get_movie_by_name(title):
    return [movie for movie in movieDetails if movie["Title"] == title]

def get_movies_by_type(mediaType):
    return [movie for movie in movieDetails if movie["Type"] == mediaType]

In [50]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "Titles" in data:
                titles_list = data["Titles"]
                for media_name in titles_list:
                    media = get_movie_by_name(media_name)
                    if media:
                        output_string += json.dumps(media, indent=4) + "\n"
                    else:
                        print(f"Error: Media '{media_name}' not found")
            elif "Type" in data:
                type_name = data["Type"]
                type_media = get_movies_by_type(type_name)
                for media in type_media:
                    output_string += json.dumps(media, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

In [51]:
print(generate_output_string(mediaResultsData))

{
    "Title": "Saw",
    "Year": "2004",
    "Rated": "R",
    "Released": "29 Oct 2004",
    "Runtime": "103 min",
    "Genre": "Horror, Mystery, Thriller",
    "Director": "James Wan",
    "Writer": "Leigh Whannell, James Wan",
    "Actors": "Cary Elwes, Leigh Whannell, Danny Glover",
    "Plot": "Two strangers awaken in a room with no recollection of how they got there, and soon discover they're pawns in a deadly game perpetrated by a notorious serial killer.",
    "Language": "English",
    "Country": "United States",
    "Awards": "9 wins & 10 nominations",
    "Poster": "https://m.media-amazon.com/images/M/MV5BM2M1MzI1MWYtYmM2ZC00OWY3LTk0ZGMtNmRkNzU1NzEzMWE5XkEyXkFqcGdeQXVyODUwOTkwODk@._V1_SX300.jpg",
    "Ratings": [
        {
            "Source": "Internet Movie Database",
            "Value": "7.6/10"
        },
        {
            "Source": "Rotten Tomatoes",
            "Value": "50%"
        },
        {
            "Source": "Metacritic",
            "Value": "46/100"


In [52]:
infoParsingSystemPrompt = f"""
You are a customer service assistant for a \
large electronic store. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.

You are a helpful assistant tasked with giving information about media.
You will be given a user query, delimited by {DELIMITER}, and data about \
media in JSON format. Use the data provided to answer the query
"""

def getCompletionFromMessages(
        messages,
        model = "gpt-4",
        temperature = 0
):
    response = client.chat.completions.create(
        messages = messages,
        temperature = temperature,
        model = model
    )
    responseContent = response.choices[0].message.content
    messages += [{"content": responseContent, "role": "assistant"}]
    return messages

def getAssistantMediaInfo(query, messages, delimiter = DELIMITER):
    processedQuery = f"{delimiter}{query}{delimiter}"
    mediaSearchMessages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": processedQuery}
    ]
    searchResultsString = getCompletionFromMessages(mediaSearchMessages)[-1]["content"]
    resultsData = json.loads(searchResultsString.replace("'", "\""))
    mediaInfoString = generate_output_string(mediaResultsData)
    processedQuery += f"\n{mediaInfoString}"
    messages += [{"role": "user", "content": processedQuery}]
    messages = getCompletionFromMessages(messages)
    print(messages[-1]["content"])
    return messages

In [53]:
messages = [{"role": "system", "content": infoParsingSystemPrompt}]

messages = getAssistantMediaInfo(user_query, messages)

The Saw series is a popular horror franchise that started in 2004. The first movie, directed by James Wan, is about two strangers who wake up in a room and discover they're part of a deadly game by a serial killer. The series continued with Saw II, III, IV, V, VI, 3D, and X, each with their own unique plot but all revolving around the same concept of a deadly game. The series also includes a short film titled "Saw Rebirth" and the original 2003 short film "Saw". 

The ratings for the movies vary, with the first Saw movie having the highest IMDb rating of 7.6/10. The movies are generally rated R and fall under the genres of Horror, Mystery, and Thriller. 

Would you like to know more about a specific movie in the series?


In [54]:
categories, flagged = createModeration(messages[-1]["content"])

for category in categories:
    print(category)

print("FLAGGED BY MODERATION:", flagged)

(('harassment', False), ('harassment', 3.754994031623937e-05))
(('harassment_threatening', False), ('harassment_threatening', 3.1695431971456856e-05))
(('hate', False), ('hate', 1.1687206097121816e-05))
(('hate_threatening', False), ('hate_threatening', 3.784017826546915e-05))
(('self_harm', False), ('self_harm', 7.021870374046557e-08))
(('self_harm_instructions', False), ('self_harm_instructions', 1.5247076134983217e-07))
(('self_harm_intent', False), ('self_harm_intent', 1.107621105944645e-08))
(('sexual', False), ('sexual', 1.4809415915806312e-05))
(('sexual_minors', False), ('sexual_minors', 5.695173967978917e-06))
(('violence', False), ('violence', 0.004747314844280481))
(('violence_graphic', False), ('violence_graphic', 0.00440045353025198))
(('self-harm', False), ('self-harm', 7.021870374046557e-08))
(('sexual/minors', False), ('sexual/minors', 5.695173967978917e-06))
(('hate/threatening', False), ('hate/threatening', 3.784017826546915e-05))
(('violence/graphic', False), ('viole

In [55]:
messages[-1]["content"]

'The Saw series is a popular horror franchise that started in 2004. The first movie, directed by James Wan, is about two strangers who wake up in a room and discover they\'re part of a deadly game by a serial killer. The series continued with Saw II, III, IV, V, VI, 3D, and X, each with their own unique plot but all revolving around the same concept of a deadly game. The series also includes a short film titled "Saw Rebirth" and the original 2003 short film "Saw". \n\nThe ratings for the movies vary, with the first Saw movie having the highest IMDb rating of 7.6/10. The movies are generally rated R and fall under the genres of Horror, Mystery, and Thriller. \n\nWould you like to know more about a specific movie in the series?'